In [ ]:
!conda create -y -n nm python==3.9
!conda activate nm
!pip install -r requirements.txt

## Local inference with DeepSparse

In [1]:
from deepsparse import Pipeline
from deepsparse.image_classification.constants import IMAGENET_LABELS

cv_pipeline = Pipeline.create(
  task='image_classification', 
  model_path='zoo:cv/classification/resnet_v1-18/pytorch/sparseml/imagenet/pruned85_quant-none-vnni',
)

/opt/conda/envs/nm/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
DeepSparse, Copyright 2021-present / Neuralmagic, Inc. version: 1.3.2 COMMUNITY | (7d31c4bf) (release) (optimized) (system=avx512, binary=avx512)
[7f251d132740 >WARN<  operator() ./src/include/wand/utility/warnings.hpp:14] Generating emulated code for quantized (INT8) operations since no VNNI instructions were detected. Set NM_FAST_VNNI_EMULATION=1 to increase performance at the expense of accuracy.


In [2]:
input_image = "cat.jpg"
inference = cv_pipeline(images=input_image)
inference

ImageClassificationOutput(labels=[282], scores=[18.798709869384766])

In [3]:
from PIL import Image
input_image = Image.open("cat.jpg")
inference = cv_pipeline(images=[input_image])
inference

ImageClassificationOutput(labels=[282], scores=[18.798709869384766])

In [4]:
labels = [IMAGENET_LABELS[label_id] for label_id in inference.labels][0]
scores = inference.scores[0]

labels, scores

('tiger cat', 18.798709869384766)

## SageMaker deployment

In [9]:
import boto3, sagemaker

region = "eu-west-1"
sm_client = boto3.client(service_name="sagemaker", region_name=region)
runtime_sm_client = boto3.client("sagemaker-runtime", region_name=region)
NAME = "neural-magic"

In [23]:
# TRYING DIFFERENT STRATEGIES

import base64, json

with open("cat.jpg", "rb") as f:
    payload = f.read()

with open("cat.jpg", "rb") as img_file:
    b64_string = base64.b64encode(img_file.read())
    payload = {"request": b64_string.decode('utf-8')}
    payload = json.dumps(payload)

response = runtime_sm_client.invoke_endpoint(
    EndpointName=NAME, 
    ContentType="application/json", #"image/x-image", 
    Accept='application/json',
    Body=payload
)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (422) from primary with message "{"detail":[{"loc":["body","request"],"msg":"field required","type":"value_error.missing"}]}". See https://eu-west-1.console.aws.amazon.com/cloudwatch/home?region=eu-west-1#logEventViewer:group=/aws/sagemaker/Endpoints/neural-magic in account 257446244580 for more information.

In [24]:
sm_client.delete_model(ModelName=NAME)
sm_client.delete_endpoint_config(EndpointConfigName=NAME)
sm_client.delete_endpoint(EndpointName=NAME)

{'ResponseMetadata': {'RequestId': '5eede8f2-d755-4417-9f59-2eef89ff0325',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5eede8f2-d755-4417-9f59-2eef89ff0325',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 11 Feb 2023 11:29:20 GMT'},
  'RetryAttempts': 0}}

In [50]:
# import base64
# with open("../cat.jpg", "rb") as img_file:
#     b64_string = base64.b64encode(img_file.read())

# predictor.predict('{"image": "'+b64_string.decode('utf-8')+'"}')